In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


In [2]:
# === 1. Load dan konversi data ===
df = pd.read_csv("mnist_test.csv")  # Ganti path jika perlu
labels = df["label"].values.astype(int)
pixels = df.drop("label", axis=1).values
images = pixels.reshape(-1, 28, 28).astype(np.float32)

In [3]:
# === 2. Ekstraksi patch acak ===
def extract_random_patches(images, patch_size=7, stride=1, max_patches=50000):
    n, h, w = images.shape
    k = patch_size
    patch_vectors = []

    for idx in range(n):
        for i in range(0, h - k + 1, stride):
            for j in range(0, w - k + 1, stride):
                patch = images[idx, i:i + k, j:j + k]
                patch = patch - np.mean(patch)
                patch_vectors.append(patch.flatten())
                if len(patch_vectors) >= max_patches:
                    return np.array(patch_vectors).T
    return np.array(patch_vectors).T

patch_matrix = extract_random_patches(images[:1000], patch_size=7, max_patches=50000)  # pakai sebagian untuk efisiensi


In [4]:
# === 3. PCA pada patch ===
pca = PCA(n_components=30, random_state=42)
pca.fit(patch_matrix.T)
filters = pca.components_.reshape(30, 7, 7)  # 30 filter PCA ukuran 7x7

In [5]:
# === 4. Transformasi fitur gambar ===
def transform_images_with_pca_filters(images, filters):
    n_images = images.shape[0]
    n_filters = filters.shape[0]
    h_out = images.shape[1] - 6
    w_out = images.shape[2] - 6
    features = np.zeros((n_images, n_filters * h_out * w_out), dtype=np.float32)

    for idx in range(n_images):
        img = images[idx]
        feats = []
        for f in filters:
            conv = convolve2d(img, f, mode='valid')
            feats.append(conv.flatten())
        features[idx] = np.concatenate(feats)
    return features

from scipy.signal import convolve2d
feature_vectors = transform_images_with_pca_filters(images, filters)

In [ ]:
# === 5. Split data dan latih SVM ===
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, stratify=labels, random_state=42)

clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Akurasi SVM: Final Test Accuracy: {acc * 100:.2f}%")

Akurasi SVM: Final Test Accuracy: 88.15%


: 